In [ ]:
import json
import os
import sys
import fnmatch


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict

from sklearn import neural_network
from sklearn import linear_model
from sklearn import preprocessing
from sklearn import feature_selection, model_selection
from sklearn import  multioutput
from sklearn import ensemble
from sklearn import svm

In [ ]:
df = pd.read_csv('simmed_stats.csv')
#df = pd.read_csv('FBGM_League_1_all_seasons_Average_Stats(1).csv')
#df = df.sample(10000)
#['QB', 'OL', 'TE', 'DL', 'S', 'CB', 'WR', 'LB', 'RB', 'P', 'K']
clf2 = None
unique_pos = df.Pos.unique()
df.shape
df.loc[(df.Pos == 'WR')&(df.AV > 25),'AV'] = 25


In [ ]:
col_list = list(df.columns)
stats_to_pred = (df.columns[col_list.index('Hgt'):col_list.index('PAc')+1])
stats_to_use = list(df.columns[col_list.index('Cmp'):col_list.index('APY')+1])

In [ ]:
need_norm = ['Cmp', 'Att', 'Yds', 'TD', 'Int', 'Sk', 'Yds.1','Rush', 'Yds.2', 'TD.1','Tgt', 'Rec', 'Yds.3', 'TD.2', 'Tch','RRTD',
 'Int.1',
 'Yds.4',
 'TD.3',
 'PD',
 'FF',
 'FR',
 'Yds.5',
 'TD.4',
 'Sk.1',
 'Tck',
 'Solo',
 'Ast',
 'TFL',
 'Sfty',
 'Fmb',
 'FG10',
 'FGA10',
 'FG20',
 'FGA20',
 'FG30',
 'FGA30',
 'FG40',
 'FGA40',
 'FG50',
 'FGA50',
 'FGM',
 'FGA',
 'XPM',
 'XPA',
 'Pnt',
 'Yds.7',
 'PR',
 'Yds.8',
 'TD.5',
 'KR',
 'Yds.9',
 'TD.6',
 'FL',
 'Pen',
 'Yds.6',
 'Blk',
'YScm',
 'APY',
 'AV']
need_norm = list(set(need_norm))

In [ ]:
BAD_SET = set(['Blk','FL', 'Pen', 'Yds.6','Lng.4', 'APY'])
need_norm = [_ for _ in need_norm if _ not in BAD_SET]

In [ ]:
for col in need_norm:
    df[col + '_PerG'] = df[col]/np.maximum(df['G'],1)
    stats_to_use.append(col + '_PerG')
for col in need_norm:
    del stats_to_use[stats_to_use.index(col)]
df.loc[(df.Pos == 'WR')&(df.AV_PerG > 25/16),'AV_PerG'] = 25/16


In [ ]:
df.shape[0]

In [ ]:
for p in df.Pos.unique():
    p ='QB'
    vec =sorted(df[(df.Pos == p)].AV_PerG)
    q1 = vec[int(round(len(vec)*0.02))]
    q99 = vec[int(round(len(vec)*0.98))]
    q1,q99
    df = df[~((df.Pos == p) & ((df.AV_PerG < q1) | ((df.AV_PerG >q99))))]
df.loc[df.AV < 0,'AV'] = 0
df.loc[df.AV_PerG <0,'AV_PerG'] = 0

df.shape[0]

In [ ]:
plt.style.use('seaborn-white')
POS = 'OL'
plt.hist(df[(df.Pos == POS)].AV)
plt.figure()
plt.hist(df[(df.Pos == POS) & (round(df.AV_PerG*16) == 5)].Ovr,10,alpha=0.5,density=True,label='AV=5')
plt.hist(df[(df.Pos == POS) & (round(df.AV_PerG*16) == 10)].Ovr,10,alpha=0.5,density=True,label='AV=10')
plt.hist(df[(df.Pos == POS) & (round(df.AV_PerG*16) == 15)].Ovr,10,alpha=0.5,density=True,label='AV=15')
plt.xlabel('overall')
plt.legend()

In [ ]:
df[(df.Pos == 'P') &(df.AV != 0)]#.AV#.to_csv('test.csv')

In [ ]:
stats_to_use = [_ for _  in stats_to_use if _ not in BAD_SET]

In [ ]:
from sklearn import ensemble
if clf2 is None:
    clf2 = linear_model.LogisticRegression(max_iter=100)
    #clf2 = ensemble.GradientBoostingClassifier(n_estimators=10)
    scalerX_pos_og = preprocessing.StandardScaler()

    fexp_pos = preprocessing.PolynomialFeatures(degree=1,interaction_only=False)
    tmpX = df[stats_to_use]
    tmpX2 = fexp_pos.fit_transform(scalerX_pos_og.fit_transform(tmpX))
    clf2.fit(tmpX2,df.Pos)
    print(clf2.score(tmpX2,df.Pos))


In [ ]:
from sklearn.pipeline import Pipeline

pos_cuts = {}
pos_models = {}

for p in df.Pos.unique():
    df_tmp = df[df.Pos == p].copy()
    df_tmp.loc[df_tmp.AV_PerG < 0,'AV'] = 0
    res = pd.qcut(df_tmp.AV_PerG,5,duplicates='drop',retbins=True)[1]
    plimits = sorted(res)+ [1000.0]
    pos_cuts[p] = plimits
    dft = df[df.Pos == p]
    #print(p,plimits)
    models = []
    for i in range(1,len(plimits)):
        dfr = dft[dft.AV_PerG <= plimits[i]]
        if dfr.shape[0] > 0:
            # simple model is
            # model = dfr[stats_to_pred].mean()
            # var = dfr[stats_to_pred].cov()
            # slight more is

            pipe = Pipeline([('fexp',preprocessing.PolynomialFeatures(1)),
                               ('scale',preprocessing.StandardScaler()),
                               ('output',multioutput.MultiOutputRegressor(linear_model.Ridge()))]) 
            # zero turns off the linear model because it seems worse
            fake_target = dfr[stats_to_pred] #np.ones_like(dfr[stats_to_pred])*np.array(dfr[stats_to_pred].mean())
            pipe.fit(np.array(dfr.AV_PerG)[:,None],fake_target)
            # get mean
            mean = pipe.predict(np.array(dfr.AV_PerG)[:,None])
            # get cov 
            cov = (dfr[stats_to_pred]- mean).cov()
            models.append([plimits[i-1],pipe,cov])
            em = np.linalg.norm(mean-dfr[stats_to_pred],axis=1).mean()
            es = np.linalg.norm(dfr[stats_to_pred].mean()-dfr[stats_to_pred],axis=1).mean()
            print(p,dfr.shape,dft.shape,plimits[i],em-es,np.linalg.slogdet(cov)[1])
        dft = dft[dft.AV_PerG > plimits[i]]
    models[0][0] = -1000
    pos_models[p] = models


In [ ]:
plt.style.use('seaborn-white')
POS = 'OL'
for p in df.Pos.unique():
    plt.figure()
    plt.hist(df[(df.Pos == p)].AV_PerG*16,density=True)
    plt.title(p)
    for lim in [_[0] for _ in  pos_models[p]][1:]:
        print(p,lim*16)
        plt.plot([lim*16,lim*16],[0,0.1],lw=4,c='r')

In [ ]:
[_[0] for _ in  pos_models[p]]

In [ ]:
GEN_YEAR = 2019
tyear = [GEN_YEAR]
all_tables = {}
for ty in tyear: 
    all_tables[ty] = np.load('fb_tables_{}.pkl'.format(ty),allow_pickle=True)
teams = all_tables[tyear[0]].keys()
print(all_tables[GEN_YEAR]['nwe'].keys())

In [ ]:
player_stats = {k:{} for k in tyear}
table_columns = defaultdict(dict)
for ty in tyear:
    tables = all_tables[ty]
    for team in tables:
        team_tables =  tables[team]
        for table_name in team_tables:
            if table_name in ['draft-rights','team_and_opponent','conf','name','logo']:
                continue
            table = team_tables[table_name].fillna(0)

            #print(table_name)
            #print(table.index)
            for row in table.itertuples():
                name = row[0]
                name = name.replace('\xa0\xa0',' ').replace('*','').replace('+','')
                if name == 'Team Totals':
                    continue
                nsplit = name.split(' ')
                if nsplit[-1] in ['Jr.','Sr.','I','II','III',"IV",'(TW)']:
                    name = ' '.join(nsplit[:-1])

                player_table = player_stats[ty].get(name,{})
                player_row = player_table.get(table_name,[])
                player_row = player_row + [row]

                player_table[table_name] = player_row
                player_table['team'] = team
                player_stats[ty][name] = player_table
                
                #if name == 'Dennis Smith Jr.' or name == 'Luka Doncic':
                #    print(player_stats[ty][name],team)
            tcs = list(table.columns)
            for i in range(len(tcs)):
                while tcs.index(tcs[i]) != i:
                    if '.' in tcs[i]:
                        tsplit = tcs[i].split('.')
                        tsplit[-1] = str(int(tsplit[-1]) + 1)
                        tcs[i] = '.'.join(tsplit)
                    else:
                        tcs[i] += '.1'
            table_columns[(ty,team)][table_name] = tcs


In [ ]:
#player_stats[2007].keys()

In [ ]:
table_mask = defaultdict(dict)
for key in table_columns:
    for table in table_columns[key]:
        table_mask[key][table] = [_.strip() !=''  for _ in table_columns[key][table] ]
        table_columns[key][table] = [_ for _ in table_columns[key][table] if _.strip() != '']

In [ ]:
for ty in tyear:
    for player in player_stats[ty]:
        team = player_stats[ty][player]['team']
        for tt in player_stats[ty][player]:
            if tt == 'team':
                continue
            new_rows = []
            for tablet in player_stats[ty][player][tt]:
                vector = [_ if _ != '' else '0.0' for _ in tablet[1:]]
                vector = [(float(_.replace('%',''))/100 if type(_) == str and'%' in _ else _) for _ in vector]
                if 'on_off' in tt:
                    vector = vector[1:]
                if 'contracts' in tt:
                    vector = vector[1:-2]
                if tt in ['salaries2','contracts']:
                    vector = [_.replace(',','').replace('$','') for _ in vector]
                try:
                    v2 = np.array(vector).astype(np.float)
                except:
                    v2 = vector
                new_rows.append(vector)
            a = np.array(new_rows)


            if tt in table_columns[(ty,team)]:
                if 'MP' in table_columns[(ty,team)][tt] and not tt in ['pbp','on_off','on_off_p']:
                    try:
                        a = a.astype(np.float)
                    except:
                        a = list(a)
                        a[0] = np.array([float(_) for _ in a[0]])
                        a[1] = np.array([float(_) for _ in a[1]])
                        a = np.array(a)
                    try:
                        mins = a[:,table_columns[(ty,team)][tt].index('MP')].reshape((-1,1))
                        new_rows = ((a.T @ mins)/mins.sum()).T
                        a = new_rows
                    except:
                        #print(tt,a.shape,player,a,mins)
                        #print('.',end='')
                        
                        print(ty,team,player,tt)
                        continue
            player_stats[ty][player][tt] = a


In [ ]:
player_stats

In [ ]:
lt = [('AV','games_played_team'),
      ('Cmp','passing'),('Att','passing'),('Pct','passing','Cmp%'),
      ('Yds','passing'),('TD','passing'),('TD%','passing'),('Int','passing'),
      ('Int%','passing'),('Lng','passing'),('Y/A','passing'),('AY/A','passing'),('QBRat','passing','Rate'),
      ('Y/C','passing'),('Y/G','passing'),('Sk','passing'),('Yds.1','passing'),('NY/A','passing'),
      ('ANY/A','passing'),('Sk%','passing'),('Rush','rushing_and_receiving','Att'),
      ('Yds.2','rushing_and_receiving','Yds'),('TD.1','rushing_and_receiving','TD'),('Lng.1','rushing_and_receiving','Lng'),
      ('Y/A.1','rushing_and_receiving','Y/A'),('Y/G.1','rushing_and_receiving','Y/G'),('A/G','rushing_and_receiving'),
      ('Tgt','rushing_and_receiving'),('Rec','rushing_and_receiving'),('Yds.3','rushing_and_receiving','Yds.1'),
      ('TD.2','rushing_and_receiving','TD.1'),('Lng.2','rushing_and_receiving','Lng.1'),('Y/R','rushing_and_receiving'),
      ('R/G','rushing_and_receiving'),('Y/G.2','rushing_and_receiving','Y/G.1'),('Ctch%','rushing_and_receiving'),
      ('Y/Tch','rushing_and_receiving'),('Y/Tgt','rushing_and_receiving'),('Tch','rushing_and_receiving','Touch'),
      ('YScm','rushing_and_receiving'),('RRTD','rushing_and_receiving'),('Int.1','defense','Int'),
      ('Yds.4','defense','Yds'),('TD.3','defense','TD'),('Lng.3','defense','Lng'),('PD','defense'),
      ('FF','defense'),('FR','defense'),('Yds.5','defense','Yds.1'),('TD.4','defense','TD.1'),('Sk.1','defense','Sk'),
      ('Tck','defense','Comb'),('Solo','defense'),('Ast','defense'),('TFL','defense'),('Sfty','defense'),('Fmb','defense'),
      ('FG10','kicking','FGM'),('FGA10','kicking','FGA'),('FG20','kicking','FGM.1'),('FGA20','kicking','FGA.1'),
      ('FG30','kicking','FGM.2'),('FGA30','kicking','FGA.2'),('FG40','kicking','FGM.3'),('FGA40','kicking','FGA.3'),
      ('FG50','kicking','FGM.4'),('FGA50','kicking','FGA.4'),('FGM','kicking','FGM.5'),('FGA','kicking','FGA.5'),
      ('Pct.1','kicking','FG%'),('XPM','kicking'),('XPA','kicking'),('Pct.2','kicking','XP%'),('Pnt','kicking'),
      ('Yds.7','kicking','Yds'),('Lng.5','kicking','Lng'),('Blk','kicking','Blck'),('Y/A.2','kicking','Y/P'),
      ('PR','returns','Ret'),('Yds.8','returns','Yds'),('TD.5','returns','TD'),('Lng.6','returns','Lng'),
      ('Y/A.3','returns','Y/R'),('KR','returns','Rt'),('Yds.9','returns','Yds.1'),('TD.6','returns','TD.1'),
      ('Lng.7','returns','Lng.1'),('Y/A.4','returns','Y/Rt')]

locations = []
for loc in lt:
    if len(loc) == 2:
        loc = list(loc)
        loc.append(loc[0])
    locations.append(loc)

        
locs = defaultdict(dict)
for ty in tyear:
    for team in all_tables[ty]:
        for loc in locations:
            try:
                locs[(ty,team)][loc[0]] = (table_columns[(ty,team)][loc[1]].index(loc[2]),loc[1])
            except Exception as e:
                #print(e)
                #print(team,ty)
                print(e,loc)
                pass#print(loc,table_columns[loc[1]])
    #if 'shooting' in table_columns:
    #    l = list(locs['LowPostFGP'])
    #    l[0] +=2
    #    locs['MidRangeFGP'] = l
len(locs[(GEN_YEAR,'nwe')]),locs[(GEN_YEAR,'nwe')]

In [ ]:
player_vectors = []
player_names = []
player_years = []
player_scales = []
player_heights = []
player_weights = []
player_drafts = []
player_ages = []
player_teams = []
player_pos = []
player_av = []
for ty in tyear:
    for name in player_stats[ty]:
        try:
            stats = player_stats[ty][name]
            team = stats['team']

            d = {}
            for k,v in locs[(ty,team)].items():
                SV  = 0
                try:
                    SV = float(stats[v[1]][0][v[0]])
                except:
                    pass
                d[k] = 0 if SV == '' else SV
            d['Pos'] = []
            for t in ['passing','defense','returns','kicking','rushing_and_receiving','games_played_team']:
                try:
                    d['Age'] = int(float(stats[t][0][table_columns[(ty,team)][t].index('Age')]))
                except:
                    pass
                try:
                    d['Pos'].append(stats[t][0][table_columns[(ty,team)][t].index('Pos')])
                except:
                    pass
                try:
                    d['G'] = max(int(stats[t][0][table_columns[(ty,team)][t].index('G')]),d.get('G',0))
                    #print(d['G'])
                except:
                    pass
            if 'games_played_team' not in stats:
                continue
            wt = stats['games_played_team'][0][4]
            ht = stats['games_played_team'][0][5]
            exp = stats['games_played_team'][0][8]
            for col in need_norm:
                if col in d:
                    d[col + '_PerG'] = d[col]/np.maximum(d.get('G',0),1)
            player_vectors.append([d[stat] for stat in tmpX.columns])
            player_names.append(name)
            player_years.append(ty)
            player_ages.append(d['Age'])
            player_av.append(d['AV_PerG'])
            player_pos.append(d['Pos'])
            player_heights.append(ht)
            player_drafts.append(exp)
            player_weights.append(wt)
            if 'Brady' in name or 'Stidham' in name:
                print(name,d['AV_PerG'],len(player_av)-1)

            player_teams.append(team)
        except Exception as e:
            print(name,team)
            print(e)
            print(d.keys())
            #if name == 'Bill Bradley':
            #    raise
            raise
tmpX.columns

In [ ]:
player_stats[ty][name]

In [ ]:
first_n = len([yr for yr in player_years if yr == tyear[0]])
gen_FA = 0#len(teams)*5 if CURRENT_YEAR!=GEN_YEAR else 0
first_n,len(teams),gen_FA
#player_names

In [ ]:
Xn = np.nan_to_num(np.array(player_vectors))
scalerX_pos = preprocessing.StandardScaler()
Xn_scale = fexp_pos.transform(scalerX_pos.fit_transform(Xn))
pred_pos = clf2.predict(Xn_scale)


In [ ]:
'QB', 'OL', 'TE', 'DL', 'S', 'CB', 'WR', 'LB', 'RB', 'P', 'K'
pos_mapping = {
 'LG':'OL',
 'RG':'OL',
 'CB':'CB',
 'LOLB':'LB',
 'RB':'RB',
 'FB':'RB',
 'K':'K',
 'C':'OL',
 'LCB':'CB',
 'RCB':'CB',
 'FS':'S',
 'RDT':'DL',
 'QB':'QB',
 'DL':'DL',
 'MLB':'LB',
 'WR':'WR',
 'S':'S',
 'SS':'S',
 'TE':'TE',
 'DT':'DL',
 'RILB':'LB',
 'T':'OL',
 'LDT':'DL',
 'P':'P',
 'LILB':'LB',
 'ROLB':'LB',
 'LB':'LB',
 'DB':'S',
 'DE':'DL',
 'NT':'DL',
 'RT':'OL',
 'LT': 'OL',
 'SE': 'WR',
 'FL': 'WR',
 'OLB':'LB',
 'DE ': 'DL',
 'OT': 'OL',
 'OL': 'OL',
 'LS': 'OL',
 'ILB':'LB',
 'G':'OL',
 'OG':'OL',
 'EDGE':'DL',
 'G,T': 'OL',
 'LS,TE': 'TE',
  'PR':'WR',
  'KR':'WR',
'RDE':'DL',
    'LDE':'DL',
        'RLB':'LB',
            'LLB':'LB'


}

In [ ]:
import random
predicted_pos = []
for listed_p,model_p in zip(player_pos,pred_pos):
    possible_pos = [_.upper() for _ in listed_p if _ != '0.0']
    pposs = []
    for pos in list(possible_pos):
        if '/' in pos:
            pp = pos.split('/')
            for ppp in pp:
                pposs.append(ppp)
        else:
            pposs.append(pos)
    possible_pos = [pos_mapping[_.replace(' ','')] for _ in set(pposs)]
    #print(possible_pos,model_p)
    
    if model_p in possible_pos:
        true_p = model_p
    elif len(possible_pos) > 0:
        random.shuffle(possible_pos)
        true_p = possible_pos[0]
    else:
        true_p = model_p
    predicted_pos.append(true_p)
predicted_pos = np.array(predicted_pos)

In [ ]:
plt.figure(figsize=(4,3))
plt.hist(player_av,20,alpha=0.5,density=True,label='true')
plt.hist(df.AV_PerG,40,alpha=0.5,density=True,label='sim')
#plt.xlim(0,32)
plt.legend()
plt.xlabel('AV')

In [ ]:
GOAL_STD = 3.5


In [ ]:
ratings = np.zeros((predicted_pos.shape[0],len(stats_to_pred)))
for pos in unique_pos:
    AVs = np.array(player_av)[np.where(predicted_pos == pos)]
    cand_set = pos_models[pos]
    gen_ratings = []
    if pos in set(['K','P']):
        AVs = AVs*0
    for n,av in zip(np.array(player_names)[np.where(predicted_pos == pos)],AVs):
        for mx,mn,cov in cand_set[::-1]:
            if av > mx:
                break

        mn = mn.predict([[av]])
        scale = (GOAL_STD**2)/np.diag(cov).mean()

        gen_ratings.append(np.random.multivariate_normal(mn[0],cov*scale))
        if ' Brady' in n or 'Stidham' in n:
            print(av,n,np.round(gen_ratings[-1])) #,mn,np.diag(cov)*scale,
    ratings[np.where(predicted_pos == pos)] = gen_ratings


In [ ]:
#all_tables[2018]['nwe'].keys()
#'logo', 'name', 'conf'
teams2 = []
for t in teams:
    try:
        aaaa = all_tables[GEN_YEAR][t]['conf'][:-9]
        teams2.append(t)
    except:
        pass
teams = teams2

In [ ]:
confs = ['AFC','NFC']
divs = list(set([all_tables[GEN_YEAR][t]['conf'][:-9] for t in teams]))
divs

In [ ]:
rost_temp = json.loads(open('roster_size.json','rt',encoding='utf-8').read())
rost_temp = {_['abbrev']:_ for _ in rost_temp['teams']}

In [ ]:
import random
base = {}

base['startingSeason'] = GEN_YEAR
valid_tids = list(range(len(teams)))

new_teams = []

for i,t1 in enumerate(sorted(list(teams))):
    t = {}
    t['abbrev'] = t1.upper()
    nsplit = all_tables[GEN_YEAR][t1]['name'].split(' ')
    t['region'] = ' '.join(nsplit[:-1])
    t['name'] = nsplit[-1]
    t['imgURL'] = all_tables[GEN_YEAR][t1]['logo']
    t['tid'] = i 
    div_name = all_tables[GEN_YEAR][t1]['conf'].rstrip().split(' ')
    t['did'] = divs.index(' '.join(div_name[:-1]))
    t['cid'] = [c in all_tables[GEN_YEAR][t1]['conf'] for c in confs].index(True)
    new_teams.append(t)
print(new_teams)
team_abbbrevs = list(teams)
    
base['teams'] = new_teams
base['gameAttributes'] = []
base['version'] = 33
DIVS = []
for div in divs:
    cid = [c in div for c in confs].index(True)
    did = divs.index(div)
    DIVS.append({'did':did, 'cid':cid, 'name':div})
print(DIVS)
base['gameAttributes'].append({'key':'divs', 'value': DIVS})
base['gameAttributes'].append({'key':'confs', 'value': [{'cid':i, 'name':n } for i,n in enumerate(confs)]})


In [ ]:
fix_lookup = {'CLT':'IND','CRD':'ARI','GNB':'GB','HTX':'HOU','KAN':'KS','NOR':'NO','NWE':'NE','OTI':'TEN','RAI':'OAK','RAM':'LA','RAV':'BAL','SDG':'LAC','SFO':"SF",'TAM':'TB'}

for n in new_teams:
    short = fix_lookup.get(n['abbrev'],n['abbrev'])
    n['abbrev'] = short
    n['pop'] = rost_temp[short]['pop']
    n['stadiumCapacity'] = rost_temp[short]['stadiumCapacity']
base['teams'] = new_teams

In [ ]:
l2 = [_.lower() for _ in stats_to_pred]
l2[1] = 'stre'
l2[3] = 'endu'
l2[-7] = 'tck'

In [ ]:
import pprint
import copy
import random
gen_player = []
pp = pprint.PrettyPrinter()
for i,name in enumerate(player_names):
    py = player_years[i]
    name = player_names[i].replace('*','').replace('+','')
    if name in ['Team Total','Opp Total']:
        continue
    sname = name.split(' ')
    new_player = {}
    new_player['firstName'] = sname[0]
    new_player['lastName'] = ' '.join(sname[1:])
    
     
    r_vec = {k: ratings[i][i2] for i2,k in enumerate(l2)}
    r_vec = {k: int(round(np.clip(v,0,100))) for k,v in r_vec.items()}
    #for t in ['hgt', 'stre', 'spd', 'jmp', 'endu', 'ins', 'dnk', 'ft', 'fg', 'tp', 'diq', 'oiq', 'drb', 'pss', 'reb']:
    #    if not t in r_vec:
    #        r_vec[t] = 50

    true_pos = [predicted_pos[i]]
    r_vec['pos'] = true_pos[0]
    new_player['pos'] =true_pos[0]
    #print(set(true_pos))


    new_player['ratings'] = [r_vec]
    age = player_ages[i]
    new_player['born'] = {'year':GEN_YEAR-age,'loc':''}
    new_player['tid'] = team_abbbrevs.index(player_teams[i])

    exp = player_drafts[i]
    try:
        exp = int(exp)
    except:
        exp = 0
    new_player['draft'] = {"originalTid": -1, "pick": 0, "round": 0, "tid": -1, "year": GEN_YEAR-exp}
    new_player['weight'] = int(float(player_weights[i]))
    try:
        ht = [int(_) for _ in player_heights[i].split('-')]
        hgt = ht[0]*12 + ht[1]
        #print(ht,hgt)
        new_player['hgt'] = int(hgt)#int(3.7*(hgt-66))
    except:
        pass
    gen_player.append(new_player)


In [ ]:
sum(['pos' in _['ratings'][0] for _ in gen_player]),len(gen_player)

In [ ]:
base['players'] = gen_player

from colorthief import ColorThief
import requests
import io
for t in base['teams']:
    color_u = ColorThief(io.BytesIO(requests.get(t['imgURL']).content)).get_palette(2)
    color_s = ['#%02X%02X%02X' % v2 for v2 in color_u]
    t['colors'] = color_s
with open('fbgm_roster_{}.json'.format(tyear[0]),'wt') as fp:
    json.dump(base,fp, sort_keys=True)

In [ ]:
len(ratings),len(ratings)
gen_player

In [ ]:
ppos = [_.upper() for _ in set(sum(player_pos,[])) if _ != '0.0']

In [ ]:
pPos = []
for p in list(ppos):
    if '/' in p:
        for pp in p.split('/'):
            pPos.append(pp)
    else:
        pPos.append(p)
pPos = list(set(pPos))

In [ ]:
for t in gen_player:
    if 'Brady' in t['lastName'] or 'Stidham' in t['lastName']:
        print(t)

In [ ]:
ratings

In [ ]:
df[(df.Pos == 'OL') & (df.YScm > 1000)].AV.mean()